# Configuration

In [1]:
import os
from tqdm.notebook import tqdm
from tqdm import tqdm
import librosa
import pandas as pd 
import pandas as np
from sklearn.metrics import classification_report

MODELS_FOLDER = "models"
SCALERS_FOLDER = "scalers"

# METTERE PATH MODELS E SCLAERS E CALCOLARE LA FINAL PREDICTION DIVERSAMENTE PRENDENDENDO PEZZI DI CODICE DELL'ENSEMBLE

In [2]:
# import utils
import feature_extraction as fe
import emotion_predictions as ep
import scipy
import numpy as np
from scipy import signal
from scipy.io.wavfile import write
#from utils import resample, denoise
import sklearn
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# print('The scikit-learn version is {}.'.format(sklearn.__version__))

# Read dataframes

In [3]:
preprocess_path = "/Users/helemanc/PycharmProjects/ambient-intelligence/notebooks_binary_model/df_csv_noise/ravdess/"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_val = pd.read_csv(os.path.join(preprocess_path,"df_val.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

In [4]:
df_test.tail()

,emotion_label,gender,path
235,fear,female,/Users/helemanc/PycharmProjects/ambient-intell...
236,fear,female,/Users/helemanc/PycharmProjects/ambient-intell...
237,angry,female,/Users/helemanc/PycharmProjects/ambient-intell...
238,surprise,female,/Users/helemanc/PycharmProjects/ambient-intell...
239,surprise,female,/Users/helemanc/PycharmProjects/ambient-intell...


In [5]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
labels= pd.Series(list(df_test.emotion_label)).replace(emotion_enc)

In [6]:
def denoise(samples):
    """
    :param samples: an array representing the sampled audio file
    :type samples: float
    :return: an array representing the clean audio file
    :rtype: float
    """
    samples_wiener = scipy.signal.wiener(samples)
    return samples_wiener

def resample(input_data, sample_rate, required_sample_rate=16000, amplify=False):
    """
    Resampling function. Takes an audio with sample_rate and resamples
    it to required_sample_rate. Optionally, amplifies volume.
    ​
    :param input_data: Input audio data
    :type input_data: numpy.ndarray
    :param sample_rate: Sample rate of input audio data
    :type sample_rate: int
    :param required_sample_rate: Sample rate to resample original audio, defaults to 16000
    :type required_sample_rate: int, optional
    :param amplify: Whether to amplify audio volume, defaults to False
    :type amplify: bool, optional
    :return: Resampled audio and new sample rate
    :rtype: numpy.ndarray, int
    """
    if sample_rate < required_sample_rate:
        resampling_factor = int(round(required_sample_rate/sample_rate, 0))
        new_rate = sample_rate * resampling_factor
        samples = len(input_data) * resampling_factor
        resampled = signal.resample(input_data, samples)
    elif sample_rate > required_sample_rate:
        resampling_factor = int(round(sample_rate/required_sample_rate, 0))
        new_rate = int(sample_rate / resampling_factor)
        resampled = signal.decimate(input_data, resampling_factor)
    else:
        resampling_factor = 1
        new_rate = sample_rate
        resampled = input_data
    if amplify and input_data.size > 0:
        absolute_values = np.absolute(resampled)
        max_value = np.amax(absolute_values)
        max_range = np.iinfo(np.int16).max
        amplify_factor = max_range/max_value
        resampled = resampled * amplify_factor
        resampled = resampled.round()
    return resampled.astype(np.float32), new_rate


def make_predictions(dataset, labels, model_name): 
    predictions = []
    model_predictions_list = []
    counter = 0
    for filepath in tqdm(dataset['path']):
        samples, sample_rate = fe.read_file(filepath)
        samples, sample_rate = resample(samples, sample_rate)
        new_samples = fe.cut_pad(samples)
        #new_filepath = "tmp.wav"
        for dirpath, dirnames, filenames in os.walk(MODELS_FOLDER):
            # iterate over the list of dirnames to load the convmodel
            # iterate over the list of filenames to get the svm model
            for model in dirnames:
                model_type = 'conv'
                parts = model.split('_')
                num_exp = parts[1]
                num_data = parts[2].split('.')[0]
                id_exp = num_exp + '_' + num_data
                if id_exp == model_name: 
                    print(id_exp)
                    model_path = os.path.join(dirpath, model)
                    conv_model = tf.keras.models.load_model(model_path)
                    for scaler in os.listdir(SCALERS_FOLDER):
                        parts = scaler.split('_')
                        num_exp = parts[1]
                        num_data = parts[2].split('.')[0]
                        id_exp_scaler = num_exp + '_' + num_data
                        if id_exp == id_exp_scaler:
                            # print("Loading scaler: ", scaler)
                            scaler_conv_model = scaler
                            mfccs = fe.mfccs_scaled(new_samples, scaler_conv_model, id_exp)
                            pred = ep.make_predictions(conv_model, model_type, mfccs, 'avg')
                            # add result to list and dictionary
                            predictions.append(pred)
                            #model_predictions[id_exp] = pred
            break
        #final_prediction, model_predictions = ensemble.ensemble(new_samples, prediction_scheme, return_model_predictions = True)

        #predictions.append(final_prediction)
        #model_predictions_list.append(model_predictions) 
        print("True label", labels[counter], "Predicted label", predictions[counter])
        counter+=1
    return predictions

# Utils Confidence

In [10]:
def compute_confidence(predictions): 
    sum_abs = 0
    for el in predictions: 
        #print(abs(el-0.5))
        sum_abs += abs(el-0.5)
        
    mean = sum_abs/len(predictions)
    var = sum([((x-mean)**2) for x in predictions])/len(predictions)
    stdev = var ** 0.5
    
    return mean, stdev 

In [11]:
def compute_confidence_square(predictions):
    sum_abs = 0
    for el in predictions: 
        #print(abs(el-0.5))
        sum_abs += (abs(el-0.5))**2
        
    mean = sum_abs/len(predictions)
    var = sum([((x-mean)**2) for x in predictions])/len(predictions)
    stdev = var ** 0.5
    
    return mean, stdev 
    

# Model 3_1

In [8]:
predictions = make_predictions(df_test, labels,'3_1')

  0%|          | 0/240 [00:00<?, ?it/s]

3_1


  0%|          | 1/240 [00:02<08:36,  2.16s/it]

True label 1 Predicted label 0.99956155
3_1


  1%|          | 2/240 [00:03<07:04,  1.78s/it]

...

True label 1 Predicted label 0.12079632
3_1


100%|█████████▉| 239/240 [03:09<00:00,  1.36it/s]

True label 0 Predicted label 0.4427958
3_1


100%|██████████| 240/240 [03:10<00:00,  1.26it/s]

True label 0 Predicted label 0.24920133


In [12]:
mean, stdev = compute_confidence(predictions)

In [13]:
mean_square, stdev_square = compute_confidence_square(predictions)

In [14]:
mean, stdev

(0.2765349503606558, 0.3648913897245413)

In [15]:
mean_square, stdev_square

(0.10658350381309445, 0.4692640462988032)

In [28]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [29]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [30]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.68      0.85      0.75       112
           1       0.83      0.65      0.73       128

    accuracy                           0.74       240
   macro avg       0.75      0.75      0.74       240
weighted avg       0.76      0.74      0.74       240



# Model 3_2

In [16]:
predictions = make_predictions(df_test, labels,'3_2')

  0%|          | 0/240 [00:00<?, ?it/s]

3_2


  0%|          | 1/240 [00:00<03:16,  1.21it/s]

True label 1 Predicted label 0.99987864
3_2


  1%|          | 2/240 [00:01<03:12,  1.24it/s]

...

True label 1 Predicted label 0.7981724
3_2


100%|█████████▉| 239/240 [03:04<00:00,  1.38it/s]

True label 0 Predicted label 0.15135589
3_2


100%|██████████| 240/240 [03:05<00:00,  1.30it/s]

True label 0 Predicted label 0.3507535


In [17]:
mean, stdev = compute_confidence(predictions)

In [18]:
mean_square, stdev_square = compute_confidence_square(predictions)

In [19]:
mean, stdev

(0.3230378472996373, 0.4259448102434205)

In [20]:
mean_square, stdev_square

(0.13030411169716635, 0.5553045793405244)

In [32]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [33]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [34]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.88      0.80      0.84       112
           1       0.84      0.91      0.87       128

    accuracy                           0.86       240
   macro avg       0.86      0.85      0.86       240
weighted avg       0.86      0.86      0.86       240



# Model 4_1

In [21]:
predictions = make_predictions(df_test, labels,'4_1')

  0%|          | 0/240 [00:00<?, ?it/s]

4_1


  0%|          | 1/240 [00:00<02:48,  1.42it/s]

True label 1 Predicted label 0.99864733
4_1


  1%|          | 2/240 [00:01<02:45,  1.44it/s]

...

True label 1 Predicted label 0.41456074
4_1


100%|█████████▉| 239/240 [03:09<00:00,  1.47it/s]

True label 0 Predicted label 0.4878242
4_1


100%|██████████| 240/240 [03:10<00:00,  1.26it/s]

True label 0 Predicted label 0.5103322


In [22]:
mean, stdev = compute_confidence(predictions)

In [23]:
mean_square, stdev_square = compute_confidence_square(predictions)

In [24]:
mean, stdev

(0.30875986826916535, 0.39508778478328)

In [25]:
mean_square, stdev_square

(0.12239930588649266, 0.5092146348180315)

In [36]:
def create_dataframe_predictions(prediction_list):
    df_predictions = pd.DataFrame(prediction_list)
    return df_predictions

def create_dataframe_res(labels, df_predictions, dataset): 
    df_res = pd.concat([labels, 
                    df_predictions, 
                    dataset.path], axis = 1, ignore_index=True, sort=False)
    new_header = []
    new_header.append('true_label')
    new_header.append('pred_label')
    new_header = new_header 
    new_header.append('path')
    df_res.columns = new_header
    return df_res

In [37]:
df_predictions = create_dataframe_predictions(predictions)
df_res = create_dataframe_res(labels, df_predictions, df_test)

In [38]:
print(classification_report(df_res.true_label, df_res.pred_label))

              precision    recall  f1-score   support

           0       0.71      0.80      0.76       112
           1       0.81      0.72      0.76       128

    accuracy                           0.76       240
   macro avg       0.76      0.76      0.76       240
weighted avg       0.76      0.76      0.76       240

